In [17]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics.pairwise import euclidean_distances

print("FIXED MODEL LOADER FOR PATIENT CLUSTERING")
print("=" * 50)

class PatientClusteringPipeline:
    """Dummy class to handle pickle loading"""
    def __init__(self, svd_model=None, clustering_model=None, rf_columns=None):
        self.svd_model = svd_model
        self.clustering_model = clustering_model
        self.rf_columns = rf_columns
    
    def predict(self, new_patient_data):
        pass  # We won't use this, just need it to exist for pickle

def load_patient_models(filename='patient_classifier_model.pkl'):
    """
    Load the saved patient clustering models
    """
    print(f"Loading models from: {filename}")
    
    try:
        # Now pickle can find the PatientClusteringPipeline class
        with open(filename, 'rb') as f:
            model_data = pickle.load(f)
        
        print("✅ Models loaded successfully!")
        
        # Extract the components we actually need
        svd_model = model_data.get('svd_model')
        
        # Try different keys for the clustering model
        clustering_model = (model_data.get('best_clustering_model') or 
                          model_data.get('kmeans_model') or 
                          model_data.get('clustering_model'))
        
        rf_columns = model_data.get('rf_columns', [])
        cluster_interpretations = model_data.get('cluster_interpretations', {})
        
        print(f"Extracted components:")
        print(f"  SVD model: {type(svd_model).__name__}")
        print(f"  Clustering model: {type(clustering_model).__name__}")
        print(f"  RF columns: {len(rf_columns)}")
        print(f"  Clusters: {len(cluster_interpretations)}")
        
        return svd_model, clustering_model, rf_columns, cluster_interpretations
        
    except Exception as e:
        print(f"❌ Loading failed: {e}")
        return None, None, None, None

def classify_patient(patient_data, svd_model, clustering_model, rf_columns, cluster_interpretations, patient_id="Unknown"):
    """
    Classify a new patient using the loaded models
    """
    print(f"\nClassifying patient: {patient_id}")
    
    # Prepare patient data
    if isinstance(patient_data, dict):
        patient_df = pd.DataFrame([patient_data])
    else:
        patient_df = patient_data.copy()
    
    # Add missing RF columns as 0
    for col in rf_columns:
        if col not in patient_df.columns:
            patient_df[col] = 0
    
    # Extract RF features
    rf_features = patient_df[rf_columns].fillna(0).values
    
    # Apply SVD transformation
    svd_features = svd_model.transform(rf_features)
    
    # Predict cluster
    predicted_cluster = clustering_model.predict(svd_features)[0]
    
    # Calculate confidence (if K-means)
    confidence = None
    if hasattr(clustering_model, 'cluster_centers_'):
        distances = euclidean_distances(svd_features, clustering_model.cluster_centers_)[0]
        assigned_distance = distances[predicted_cluster]
        other_distances = [d for i, d in enumerate(distances) if i != predicted_cluster]
        min_other = min(other_distances) if other_distances else assigned_distance
        confidence = 1 - (assigned_distance / (assigned_distance + min_other))
    
    # Get cluster info
    cluster_info = cluster_interpretations.get(predicted_cluster, {})
    
    # Count active conditions
    active_conditions = int(rf_features.sum())
    
    result = {
        'patient_id': patient_id,
        'predicted_cluster': predicted_cluster,
        'cluster_phenotype': cluster_info.get('phenotype', 'Unknown'),
        'confidence_score': confidence,
        'active_conditions_count': active_conditions,
        'cluster_size': cluster_info.get('size', 'Unknown')
    }
    
    print(f"Result: Cluster {predicted_cluster} - {result['cluster_phenotype']}")
    if confidence:
        print(f"Confidence: {confidence:.3f}")
    
    return result


svd_model, clustering_model, rf_columns, cluster_interpretations = load_patient_models()

if all([svd_model, clustering_model, rf_columns]):
    print("\n✅ SUCCESS! Models loaded correctly")
    
    # Show detailed cluster summary with RF conditions
    print(f"\n📋 DETAILED CLUSTER SUMMARY")
    print("=" * 70)
    
    for cluster_id, info in cluster_interpretations.items():
        print(f"\n🏥 CLUSTER {cluster_id}")
        print("-" * 40)
        print(f"   Clinical Phenotype: {info.get('phenotype', 'Unknown')}")
        print(f"   Size: {info.get('size', 'Unknown')} patients", end="")
        if 'percentage' in info:
            print(f" ({info['percentage']:.1f}%)")
        else:
            print()
        
        if 'avg_age' in info and info['avg_age'] is not None:
            print(f"   Average Age: {info['avg_age']:.1f} years")
        if 'must_score' in info and info['must_score'] is not None:
            print(f"   Average MUST Score: {info['must_score']:.2f}")
        if 'weight_loss' in info and info['weight_loss'] is not None:
            print(f"   Average Weight Loss: {info['weight_loss']:.1f}%")
        
        # Display top RF conditions (>10% prevalence)
        top_conditions = info.get('top_rf_conditions', {})
        if top_conditions:
            print(f"   Top RF Conditions (>10% prevalence):")
            for rf_condition, prevalence in top_conditions.items():
                # Clean up the condition name
                clean_name = rf_condition.replace('RF_', '').replace('_', ' ').title()
                print(f"     • {clean_name:<30}: {prevalence:.1%}")
        else:
            print(f"   Top RF Conditions: No high-prevalence conditions (relatively healthy)")
    
    # Test with example patient
    print(f"\n" + "="*50)
    print("TESTING WITH EXAMPLE PATIENTS")
    print("="*50)
    
    # Example 1
    test_patient_1 = {
        'RF_dementia': 1,
        'RF_confusion': 1,
        'RF_falls': 1
    }
    
    result_1 = classify_patient(test_patient_1, svd_model, clustering_model, rf_columns, cluster_interpretations, "TEST_001")
    
    # Example 2
    test_patient_2 = {
        'RF_heart_disease': 1,
        'RF_diabetes': 1,
        'RF_hypertension': 1
    }
    
    result_2 = classify_patient(test_patient_2, svd_model, clustering_model, rf_columns, cluster_interpretations, "TEST_002")
    
else:
    print("❌ Failed to load models. Check file path and format.")

FIXED MODEL LOADER FOR PATIENT CLUSTERING
Loading models from: patient_classifier_model.pkl
✅ Models loaded successfully!
Extracted components:
  SVD model: TruncatedSVD
  Clustering model: KMeans
  RF columns: 460
  Clusters: 2

✅ SUCCESS! Models loaded correctly

📋 DETAILED CLUSTER SUMMARY

🏥 CLUSTER 0
----------------------------------------
   Clinical Phenotype: Mobility-Dominant Patient Profile
   Size: 91 patients (50.8%)
   Average Age: 86.7 years
   Average MUST Score: 1.53
   Average Weight Loss: 9.7%
   Top RF Conditions (>10% prevalence):
     • Mobility And Care Dependency  : 100.0%
     • Pain                          : 98.9%
     • Reduced Mobility              : 98.9%
     • Dry Skin                      : 84.6%
     • Nail Abnormality              : 82.4%

🏥 CLUSTER 1
----------------------------------------
   Clinical Phenotype: Pain-Dominant Patient Profile
   Size: 88 patients (49.2%)
   Average Age: 85.9 years
   Average MUST Score: 0.75
   Average Weight Loss: 6.